# Sampling (RA, dec)

LightCurveLynx provides multiple mechanisms for sampling (RA, dec). In this notebook we discuss several of the approaches and their relative tradeoffs.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from lightcurvelynx.math_nodes.ra_dec_sampler import (
    ApproximateMOCSampler,
    ObsTableRADECSampler,
    ObsTableUniformRADECSampler,
    UniformRADEC,
)
from lightcurvelynx.obstable.obs_table import ObsTable
from lightcurvelynx.obstable.opsim import OpSim

## Uniform Sampling

The simplest sampling approach is to uniformly sample (RA, dec) from the unit sphere. The `UniformRADEC` node does exactly this.

In [ ]:
uniform_sampler = UniformRADEC(node_label="uniform")
(ra, dec) = uniform_sampler.generate(num_samples=500)

plt.scatter(ra, dec, s=1)
plt.show()

However this approach has limited use when simulating a specific survey. Depending on the survey's coverage, a significant number of (RA, dec) points may fall outside the viewing area.

## Sampling from a Survey

We can sample (RA, dec) coordinates from a survey (an `ObsTable` object) in two ways. First we could sample a pointing from the survey and then a point from that field of view. Second, we could sample uniformly from the region coverage by the survey.

We consider each of these approaches below.

### Sampling Pointings

Sampling pointings from the survey provides a visit-weighted sampling of positions covered by the survey. For concreteness let's start with a survey that visits two fields: one centered at (45.0, -15.0) and the other at (315.0, 15.0). The first field is visited once and the second field is visited four times on four consecutive nights.

In [ ]:
values = {
    "observationStartMJD": np.array([0.0, 1.0, 2.0, 3.0, 4.0]),
    "fieldRA": np.array([45.0, 315.0, 315.0, 315.0, 315.0]),
    "fieldDec": np.array([-15.0, 15.0, 15.0, 15.0, 15.0]),
    "zp_nJy": np.ones(5),
}
opsim = OpSim(values, radius=30.0)

We can sample from these pointings using the `ObsTableRADECSampler` node.

In [ ]:
pointing_sampler = ObsTableRADECSampler(opsim, radius=30.0, node_label="opsim")
(ra, dec, time) = pointing_sampler.generate(num_samples=100)

plt.scatter(ra, dec, s=1)
plt.show()

As we can see, the field centered in the Northern hemisphere is sampled significantly more than the one centered in the Southern hemisphere.

### Sampling Survey Coverage

If we instead would like to sample uniformly from the area covered by the survey, we have two options (`ObsTableUniformRADECSampler` and `ApproximateMOCSampler`) with different trade offs. 

**ObsTableUniformRADECSampler** 

The `ObsTableUniformRADECSampler` uses rejection sampling to generate positions. This means that for every sampled position it randomly guesses an (RA, dec) then checks if that point falls within the survey. If not, it repeats the process until it finds a valid point or reaches `max_iterations` iterations (at which point it returns the last sample). This approach works well for surveys with significant coverage.

In [ ]:
coverage_sampler1 = ObsTableUniformRADECSampler(opsim, radius=30.0, node_label="coverage1")
(ra, dec) = coverage_sampler1.generate(num_samples=100)

plt.scatter(ra, dec, s=1)
plt.show()

The `ObsTableUniformRADECSampler` does not perform well for surveys with small coverage. In the best case it may take many guesses to get find a point within the survey. In the worst, case all `max_iterations` fail and the sampler returns a random point outside the survey.

In [ ]:
values = {
    "time": np.array([0.0]),
    "ra": np.array([10.0]),
    "dec": np.array([0.0]),
    "zp": np.ones(1),
}
ops_data = ObsTable(values)

coverage_sampler2 = ObsTableUniformRADECSampler(ops_data, radius=1.0, node_label="coverage2")
(ra, dec) = coverage_sampler2.generate(num_samples=100)

plt.scatter(ra, dec, s=1)
plt.plot(10.0, 0.0, "rx")
plt.show()

As you can see, many of the points land outside the 1 degree radius around the center of the pointing (10, 0).

**ApproximateMOCSampler**

The `ApproximateMOCSampler` samples from the area covered by a [Multi-Order Coverage Map (MOC)](https://www.ivoa.net/documents/MOC/20190215/WD-MOC-1.1-20190215.pdf), which is a collection of healpix pixels representing an area on the sky. Users can generate custom MOCs for hypothetical surveys, build a MOC from a survey, or use a helper function to create the sampler directly from the survey.

We show an example using the `from_obstable` helper function.

In [ ]:
coverage_sampler3 = ApproximateMOCSampler.from_obstable(
    ops_data,
    radius=1.0,
    node_label="coverage3",
    depth=14,
)
(ra, dec) = coverage_sampler3.generate(num_samples=100)

plt.scatter(ra, dec, s=1)
plt.plot(10.0, 0.0, "rx")
plt.show()

as we can see the sample points all fall within 1.0 degree of the center of the pointing (10, 0).

This sampling is approximate, because depending on the depth of the MOC, the healpix tiles used to cover the survey might include area outside the survey. You can tradeoff accuracy with compute cost (run time and memory) using the `depth` parameter. (Note that is you generate the MOC separately, you will need to define a max depth during that operation as well).

For example, if we use depth=8, the survey's coverage is approximated by a grid of only 786,432 pixels over the entire sky with (an average pixel width of around 14 arc minutes). We recommend at least a depth of 12 (average pixel width around 50 arc seconds) for reasonable accuracy.

As a concrete example, let's look at what happens if we prebuild the MOC at depth=4 (very coarse). Although the sampler uses a depth of 14, it cannot extract any more resolution from the input than it was given (a depth=4 MOC).

In [ ]:
# Manually build a very coarse MOC.
moc = ops_data.build_moc(radius=1.0, max_depth=4)
coverage_sampler3 = ApproximateMOCSampler(moc, depth=14, node_label="coverage3")
(ra, dec) = coverage_sampler3.generate(num_samples=100)

plt.scatter(ra, dec, s=1)
plt.plot(10.0, 0.0, "rx")
plt.show()

Because of the coarse approximation, the sampled data now contains many points outside the survey's radius.

Both the manual construction of a MOC from the `ObsTable` and the `ApproximateMOCSampler.from_obstable` helper function can account for the detector footprint (if one is provided) by setting the argument `use_footprint=True`.

## Linking (RA, dec) to Models

To be useful, the (RA, dec) locations that we generate must be linked into our model objects. To support this all the generators above produce a pair of named outputs "ra" and "dec". This means we can use LightCurveLynx's reference functionality to set the object's position based on the samples.

In [ ]:
from lightcurvelynx.models.basic_models import ConstantSEDModel

model = ConstantSEDModel(
    ra=uniform_sampler.ra,
    dec=uniform_sampler.dec,
    brightness=100.0,
    node_label="source",
)
state = model.sample_parameters(num_samples=10)
print(state)

## Positions Outside the Surveys

When the positional sampler includes positions outside the survey's footprint, the simulation will occasionally draw an (RA, dec) that is not observed at any times. As a result, no data will be simulated and the results table will include a row with an empty light curve. This is the most common reason for empty light curves. The user can either refine the bounds of the sampler (such as increasing the MOC depth) or just filter out the unobserved points.